In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.datasets import make_regression

## Исходные данные

In [3]:
# сгенерируем данные
classification_data, classification_labels = make_classification(n_features=2, n_informative=2, 
                                                                 n_classes=2, n_redundant=0,
                                                                 n_clusters_per_class=1, random_state=5, n_samples=1000)
#classification_data, classification_labels = make_circles(n_samples=30, random_state=6)

In [77]:
#исходные данные для регресии
regression_data, regression_labels = make_regression(n_features=2, 
                                                     n_informative=2, 
                                                     n_targets = 1,
                                                     noise = 1.25,
                                                     random_state=6, 
                                                     n_samples=100)

## Домашнее задание

**1. В коде из методички реализуйте один или несколько из критериев останова (количество листьев, количество используемых признаков, глубина дерева и т.д.)**

### Реализация дерева решений

In [5]:
def print_tree(node, spacing=""):

    # Если лист, то выводим его прогноз
    if isinstance(node, Leaf):
        print(spacing + "Прогноз:", node.prediction)
        return

    # Выведем значение индекса и порога на этом узле
    print(spacing + 'Индекс', str(node.index), '<=', str(node.t))

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Рекурсионный вызов функции на отрицательном поддереве
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [6]:
# Расчет прироста

def gain(left_labels, right_labels, root_gini):

    # доля выборки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return root_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

In [7]:
# Расчет критерия Джини

def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

In [8]:
# Реализуем класс узла

class Node: 
    """Clss implements single tree node"""
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле
       

In [9]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
            
        # найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction  

In [10]:
# Разбиение датасета в узле

def split(data, labels, column_index, t):
    
    left = np.where(data[:, column_index] <= t)
    right = np.where(data[:, column_index] > t)
        
    true_data = data[left]
    false_data = data[right]
    
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [11]:
def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [12]:
def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [13]:
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

Разделение датасетов

In [14]:
X_train, X_test, y_train, y_test = train_test_split(classification_data, 
                                                                    classification_labels, 
                                                                    test_size=0.3,
                                                                    random_state=7)

In [15]:
X_train.shape[0]

700

min_samples_leaf - уже было представлено в коде. Таким образм для реаизации этого разграничения достаточно было раскомментировать соответсвующие строки кода в find_best_split функции.  
max_depth - в качестве вкличины глубины бралось количество узлов не считая RootNode. соответсвенно, дерево с глубиной 0 в данной реализации имеет только один Root Node.  
min_samples_split - количество узлов при котором разрешен дальнейший сплит.

Функции, которые были изменены в рамках этой задачи

In [16]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels, min_samples_leaf):
    """min samples leaf has been added as parameter
    """
    root_gini = gini(labels)

    best_gain = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique(data[:, index])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_samples_leaf or len(false_data) < min_samples_leaf:
                continue
            
            current_gain = gain(true_labels, false_labels, root_gini)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_gain > best_gain:
                best_gain, best_t, best_index = current_gain, t, index

    return best_gain, best_t, best_index

In [17]:
# Построение дерева с помощью рекурсивной функции
# Добавлены новые параметры к вызову главной функции для условий необходимых при выполнении ДЗ
def build_tree(data, labels, depth = 0 ,min_samples_leaf = 1, min_samples_split=2):
    
    #-------------------------------------------
    # используем глобальную переменную для передачи параметра максимальной глубины дерева
    global tree_max_depth
    #print(depth)
    # Прекращение рекурсии при достижении максимальной глубины дерева 
    # или если длина массива с данными меньше определенной величины min_samples_split
    if (depth < tree_max_depth) and (data.shape[0] > min_samples_split):
        gain, t, index = find_best_split(data, labels, min_samples_leaf)
    #--------------------------------------------
        
        #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
        if gain == 0:
            return Leaf(data, labels)
    

        true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

        depth += 1
        # Рекурсивно строим два поддерева
        true_branch = build_tree(true_data, true_labels, depth, min_samples_leaf, min_samples_split)

        false_branch = build_tree(false_data, false_labels, depth, min_samples_leaf, min_samples_split)
    
        # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
        return Node(index, t, true_branch, false_branch)
    
    # returning leaf when reaches max depth
    return Leaf(data, labels)
    

Проверка работы доработанного дерева.

In [18]:
tree_max_depth = 5

In [19]:
my_tree = build_tree(X_train, y_train, min_samples_leaf=1, min_samples_split = 2)

In [20]:
y_pred_train = predict(X_train, my_tree)
y_pred_test = predict(X_test, my_tree)

In [21]:
train_accuracy = accuracy_metric(y_train, y_pred_train)
train_accuracy

97.42857142857143

In [22]:
test_accuracy = accuracy_metric(y_test, y_pred_test)
test_accuracy

95.66666666666667

In [23]:
print_tree(my_tree)

Индекс 0 <= 0.11262584256582331
--> True:
  Индекс 1 <= -1.328019471718083
  --> True:
    Индекс 1 <= -1.655622580730638
    --> True:
      Индекс 0 <= -1.3947317874033134
      --> True:
        Прогноз: 1
      --> False:
        Индекс 0 <= -0.8728986138474495
        --> True:
          Прогноз: 0
        --> False:
          Прогноз: 0
    --> False:
      Индекс 0 <= -0.40624967911194587
      --> True:
        Индекс 1 <= -1.5754574590319361
        --> True:
          Прогноз: 1
        --> False:
          Прогноз: 0
      --> False:
        Индекс 1 <= -1.5651256447341462
        --> True:
          Прогноз: 0
        --> False:
          Прогноз: 1
  --> False:
    Прогноз: 0
--> False:
  Индекс 1 <= -1.4518330557811816
  --> True:
    Прогноз: 0
  --> False:
    Индекс 1 <= -0.9250008192636487
    --> True:
      Индекс 1 <= -0.9253697821973186
      --> True:
        Индекс 1 <= -1.0626111898166881
        --> True:
          Прогноз: 1
        --> False:
          Прогн

При установке min_samples_leaf > 35 дерево вырождается.


Maximum number of leaf nodes  
Maximum features to consider for a split

**2. *Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений.**

Доработанная функция.  
Критерий Джини заменен на расчет дисперсии приведенный в отдлельной функции.  
Результат рассчитывается, как сумма дисперсий для следующих ветвей.

In [88]:
# Доработанная функция расчета дисперсии

def variance_regr(left_labels, right_labels, root_variance):

    general_variance = (left_labels.shape[0] * variance(left_labels) + right_labels.shape[0] 
                        * variance(right_labels))/(left_labels.shape[0] + right_labels.shape[0])
    #result = root_variance - general_variance
    result = general_variance
    
    return result

стандартный расчет дисперсии.  
Может быть заменен встроенной функцией

In [25]:
def variance(labels):
    # расчет дисперсии для переденного массива
    branch_variance = (np.power((labels - labels.mean()), 2)).sum()/labels.shape[0]
    # ну, или labels.var()  :)

    return branch_variance

Класс содержит небольшие переименования, чтобы не пересекаться с предыдущим заданием.

In [26]:
# Доработанный класс терминального узла (листа)

class Leaf_regr:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict_regr()
        
    def predict_regr(self):
        #Найдем среднее по всем значениям и вернем его в качестве предсказания.
        prediction = self.labels.mean()
        return prediction  

Доработанный best_split.  
Адаптировано к дисперсии  
Дополнительно добавлено ограничение глубины дерева из предыдущей задачи.

In [71]:
# Нахождение наилучшего разбиения. Изменение критерия gini на дисперсию вместе с переименованием переменных.

def find_best_split_regr(data, labels):
    
    #  обозначим минимальное количество объектов в узле
    min_samples_leaf = 5

    root_variance = variance(labels)

    best_variance = np.inf
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique(data[:, index])        
     
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split_regr(data, labels, index, t)

            #  пропускаем разбиения, в которых в узле остается меньше определенного количество объектов
            if len(true_data) < min_samples_leaf or len(false_data) < min_samples_leaf:
                continue
            
            current_variance = variance_regr(true_labels, false_labels, root_variance)
                        
            #  выбираем порог, на котором получается минимальная дисперсия
            if current_variance < best_variance:                
                best_variance, best_t, best_index = current_variance, t, index
                #print(best_variance, best_t, best_index)
    
    #print(best_variance, best_t, best_index)
    return best_variance, best_t, best_index

Т.к. мы исключили критерий Джини, то имеет смысл оптимизировать разделение на ветви по величине дисперсии, а не по информационному критерию.

In [69]:
# Построение дерева с помощью рекурсивной функции

def build_tree_regr(data, labels, depth = 0):
    
    global tree_max_depth_regr
    global epsilon
    
    if (depth < tree_max_depth_regr):    
        
        variance, t, index = find_best_split_regr(data, labels)
        #print("Current data", variance, t, index)
        
        if variance == 0 or (variance == np.inf and t is None and index is None):
            return Leaf_regr(data, labels)
        
        #print("split from build_tree")
        true_data, false_data, true_labels, false_labels = split_regr(data, labels, index, t)

        # Рекурсивно строим два поддерева
        #print(true_data.shape, true_labels.shape)
        
        depth += 1
        
        true_branch = build_tree_regr(true_data, true_labels, depth)
    
        false_branch = build_tree_regr(false_data, false_labels, depth)    
     
        # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
        return Node(index, t, true_branch, false_branch)
    
    return Leaf_regr(data, labels)

Изменено название класса

In [29]:
def classify_object_regr(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf_regr):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object_regr(obj, node.true_branch)
    else:
        return classify_object_regr(obj, node.false_branch)

Изменено только название класса.

In [30]:
def predict_regr(data, tree):    
    classes = []
    for obj in data:
        prediction = classify_object_regr(obj, tree)
        classes.append(prediction)
    return classes

### Классы и функции без изменений

In [32]:
# Реализуем класс узла

class Node: 
    """Clss implements single tree node"""
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

Изменено только название функции для исключения пересечения с предыдущим заданием

In [33]:
# Разбиение датасета в узле

def split_regr(data, labels, column_index, t):
 
    left = np.where(data[:, column_index] <= t)
    right = np.where(data[:, column_index] > t)
        
    true_data = data[left]
    false_data = data[right]
    
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

Функция расчета среднеквадратичной ошибки, как метрики для регрессии

In [84]:
def mse(y, y_pred):
    
    mse_value = (np.sum((y_pred - y)**2)) / y.shape[0]
    
    return mse_value
    

### Проверка работы доработанного дерева

Разделение датасета

In [89]:
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(regression_data, 
                                                                    regression_labels, 
                                                                    test_size=0.3,
                                                                    random_state=7)

Глобальные переменные - ограничение параметров дерева

In [90]:
tree_max_depth_regr = 10
epsilon = 1.05

In [91]:
my_tree_regr = build_tree_regr(X_train_reg, y_train_reg)

In [92]:
y_pred_train_regr = predict_regr(X_train_reg, my_tree_regr)
y_pred_test_regr = predict_regr(X_test_reg, my_tree_regr)
#y_pred_train_regr

Сревнение величины ошибки для тренировочного и тестового датасетов

In [93]:
mse_train = mse(y_train_reg, y_pred_train_regr)
mse_train

651.3144590245006

In [94]:
mse_test = mse(y_test_reg, y_pred_test_regr)
mse_test

820.0182152431594